In [55]:
import requests
import pandas as pd
import urllib3
import json
import datetime


def string_to_dict(dict_string):
    # Convert to proper json format
    dict_string = dict_string.replace("T00:00:00", '')
    tmp_dict = json.loads(dict_string)
    new_ord = ['date','close','stockAgentMainPower','stockAgentDiff','skp5','skp20']
    ret_dict = {}
    for k in new_ord:
        if(k=='date'):
            data = datetime.datetime.strptime(tmp_dict.get(k), '%Y-%m-%d')
        else:
            data = tmp_dict.get(k)
        ret_dict.update({k:data})
    return ret_dict


def download_major_investors(stockid):
    urllib3.disable_warnings()
    url = 'https://www.wantgoo.com/stock/%s/major-investors/main-trend-data' %stockid
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36'}

    
    r = requests.get(url, headers=headers)
    a = r.text[1:len(r.text)-1]
    tmp_list = []
    while(len(a)>0):
        n = a.find('},')
        if(n>0):
            b = a[:n+1]
    #         print(b)
            tmp_list.append(b)
            c = a[n+2:]
            a = c
        else:
            n=a.find('}')
            b = a[:n+1]
    #         print(b)
            tmp_list.append(b)
            c = a[n+2:]
            a = c

    major_list=[]
    for ls in tmp_list:
        major_list.append(string_to_dict(ls))

    cols = {'date':'日期','close':'收盤價','stockAgentMainPower':'買賣超','stockAgentDiff':'家數差','skp5':'5日集中(%)','skp20':'20日集中(%)'}
    df = pd.DataFrame(major_list)
    df.rename(columns=cols, inplace=True)
    df.set_index('日期', inplace=True)
    return df

def main():
    mystock='4906'
    dfmajor_investors=download_major_investors(mystock)
    display(dfmajor_investors)


if __name__ == "__main__":
    main()

,收盤價,買賣超,家數差,5日集中(%),20日集中(%)
日期,,,,,
2020-11-05,28.50,-3127,301,-3.5426,-0.9930
2020-11-04,28.85,-1069,123,-1.3756,0.2501
2020-11-03,29.20,-13194,480,-1.5631,0.7696
2020-11-02,30.50,7789,-244,11.2201,9.7326
2020-10-30,27.75,2851,-384,10.2915,9.7471
...,...,...,...,...,...
2019-11-12,24.00,-181,59,-15.3472,-18.4646
2019-11-11,23.75,-502,171,-19.7496,-17.8328
2019-11-08,24.40,6,12,-9.4975,-17.8391
